In [1]:
trait Debug {
    def debug(): String
}

defined trait Debug

In [2]:
trait Wallet[T] {
    def load(filename: String)
    def load(is: java.io.InputStream)
    def load(s: scala.collection.immutable.Stream[String])
    def balance(): T
    def spend(quantity: T): T
}

defined trait Wallet

In [3]:
trait WalletData[T] {
    protected var wallet: Seq[T]
}

defined trait WalletData

In [4]:
trait WalletImpl extends Wallet[Long] with Debug {
    self: WalletData[Long] =>
    
    def load(filename: String) = load(new java.io.FileInputStream(filename))
    def load(is: java.io.InputStream) = load(scala.io.Source.fromInputStream(is).getLines.toStream)
    def load(stream: scala.collection.immutable.Stream[String]) = {
        wallet = stream
            .map(line => line.toLong)
            .toList
    }
    
    def balance(): Long = wallet.sum
    
    def spend(quantity: Long) : Long = {
        var consumed = 0L
        var p = wallet
        while(!p.isEmpty && (consumed < quantity)) {
            consumed = consumed + p.head
            if(consumed <= quantity) {
                p = p.tail
            } else {
                p = (consumed - quantity) +: p.tail
            }
        }
        if(consumed < quantity) 0L else {
            wallet = p
            quantity
        }
    }
    
    def debug = wallet.toString
}

defined trait WalletImpl

In [5]:
val w = new { var wallet = Seq.empty[Long] } with WalletData[Long] with WalletImpl
w.load("data/wallet/wallet.sample.txt")

w: AnyRef with WalletData[Long] with WalletImpl = $sess.cmd4Wrapper$Helper$$anon$1@6967eb78

In [6]:
w.debug
w.balance

res5_0: String = "List(1, 100, 200, 200, 1000, 1)"
res5_1: Long = 1502L

In [7]:
w.spend(702)
w.debug
w.balance

res6_0: Long = 702L
res6_1: String = "List(799, 1)"
res6_2: Long = 800L

In [8]:
w.spend(702)
w.debug
w.balance

res7_0: Long = 702L
res7_1: String = "List(97, 1)"
res7_2: Long = 98L

In [9]:
w.spend(702)
w.debug
w.balance

res8_0: Long = 0L
res8_1: String = "List(97, 1)"
res8_2: Long = 98L